In [7]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1012. Allostatic Load/P1012.3. Analyses/P1012.3.2. Ordinal Regression/Output')
cur_date = "052324"

library(readxl)
library(openxlsx)
library(tidyverse)
library(ordinalNet)

# reading in file
#cytokine_df = data.frame(read_excel("Input/Allostatic_Mediator_Data_050824.xlsx", sheet = 2)) 
bp_df = data.frame(read_excel("Input/Allostatic_Mediator_Data_050824.xlsx", sheet = 4)) 
subject_info_df = data.frame(read_excel("Input/Subject_Info_050824.xlsx", sheet = 2))
allostatic_load_df = data.frame(read_excel("Input/Allostatic_Load_BP_Subjects_051324.xlsx"))

In [8]:
#head(cytokine_df)
head(bp_df)
head(subject_info_df)
head(allostatic_load_df)

,Subject_ID,Systolic,Diastolic,BP_Classification
,<chr>,<dbl>,<dbl>,<chr>
1,CS_26,133,86,1
2,CS_29,116,84,N
3,CS_17,118,80,N
4,NS_63,122,85,E
5,CS_21,112,77,N
6,CS_27,131,91,1


,Study,Original_Subject_Number,Subject_Number,Group,Subject_ID,Sex,Age,Race
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>
1,FM2,97,1,CS,CS_1,M,18,B
2,FM2,99,2,CS,CS_2,F,20,W
3,FM2,101,3,CS,CS_3,F,20,W
4,FM2,98,4,CS,CS_4,F,21,W
5,TCORS LAIV,39,5,CS,CS_5,M,21,W
6,FM2,100,6,CS,CS_6,F,22,W


,Group,Covariate,Subject_ID,Allostatic_Load
,<chr>,<chr>,<chr>,<dbl>
1,NS,B,NS_50,3.075079
2,CS,B,CS_24,2.982104
3,CS,NA,CS_32,2.902104
4,NS,W,NS_49,2.873066
5,CS,W,CS_31,2.832494
6,NS,W,NS_38,2.815190


In this analysis, we're using [ordinal regression](https://www.statisticssolutions.com/free-resources/directory-of-statistical-analyses/ordinal-regression/) to predict blood pressure classification using allostatic load scores and ???. 

In [9]:
# creating 1 df
full_df = inner_join(bp_df[,c(1,4)], subject_info_df[,4:8]) %>%
    # filtering for subjects who have blood pressure measurements
    #filter(Subject_ID %in% unique(bp_df$Subject_ID)) %>%
    # adding allostatic load data
    inner_join(allostatic_load_df) %>%
    # only keep overall allostatic load scores   
    filter(if_any(Covariate, is.na)) %>%
    select(-Covariate) 

# turning BP into a factor
full_df$BP_Classification = factor(full_df$BP_Classification, levels = c("N", "E", "1", '2'))

full_df = full_df[,c(1,3:7,2)]
    

head(full_df)

Joining with `by = join_by(Subject_ID)`
Joining with `by = join_by(Subject_ID, Group)`


,Subject_ID,Group,Sex,Age,Race,Allostatic_Load,BP_Classification
,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<fct>
1,CS_26,CS,F,34,B,0.6931927,1
2,CS_29,CS,F,36,B,0.7449297,N
3,CS_17,CS,M,29,W,1.8285318,N
4,NS_63,NS,M,43,B,1.0934833,E
5,CS_21,CS,F,31,B,1.8805092,N
6,CS_27,CS,F,35,W,0.6372032,1


# Ordinal Regression

dataset is a data frame containing the sample names under the first column, the feature data in the next n-1 columns, and the response data in the last column under response. This order matters for line 25, where you get a matrix of just the features. Could be obtained in better ways.
Line 27 runs the actual regression using the matrix of features, and a factor of responses, both having the same order of samples they were obtained from.

In [19]:
############## Start Weight Estimation Process #####################
estimateOrdWeights <- function(dataset, response){
    # ADD WORDS
    # ordinal regression or estimating ordinal weights
    
    ############# get required Regression Parameters
    
    #get number of response levels
    NoBins <- length(unique(dataset[[response]]))
    print(NoBins)
    #get number of features or slieces
    noSlices <- ncol(dataset) - 2
    
    #set ordering of response levels as factor
    # orderedBins <- vector()
    # for(i in 1:NoBins){
    #   orderedBins <- c(orderedBins, as.character(i))
    # }
    # Bins_factor <- factor(dataset[,'response'], ordered = TRUE, levels = orderedBins)
    trues <- rep(TRUE, noSlices)
    print(trues)
    
    #run the regression
    feature_data <- as.matrix(dataset[,c(2:(ncol(dataset)-1))])
    #class(featuredata) <- "numeric"

    # ordinal regression
    # positiveID specifies whether or not each coefficient for each ind. variable should be constrained to be non-negative
    regression_model <- ordinalNet(feature_data, dataset[[response]], positiveID = trues, 
                            #?
                            #?
                         lambdaVals = 0, maxiterOut = 100)
    testW <- regression_model$coefs[NoBins:(noSlices + NoBins - 1)]
    print(testW)
    testW <- testW/sum(testW)
    
    return(testW)
}

In [20]:
# calling fn
estimateOrdWeights(full_df[,c(1,6,7)], 'BP_Classification')

[1] 4
[1] TRUE
[1] 0


[1] NaN

In [5]:
############## Create Testing Function #########################################
testWeights <- function(data, dataset, w, observedResponse, binRanks){
  idName <- 'Name'
  tmpData <- as.matrix(data[,-1])
  class(tmpData) <- "numeric"
  score <- rowSums(tmpData*rep(w, each = NROW(tmpData)), na.rm = TRUE)
  testData <- cbind(data,score)
  
  #sort the data based on new scores
  testData <- testData[order(testData[,'score'], decreasing = TRUE),]
  
  #determine the rank and responses of "known" profiles using the proposed weights
  noSamples <- nrow(data)
  noKnown <- nrow(dataset)
  rank <- c(1:noSamples)
  testData <- cbind(testData, rank)
  testData <- cbind(testData, observedResponse)
  testKnowns <- testData[testData[,idName] %in% dataset[,idName],]
  
  #get fitness of weights based on deviation of bins*(indices from bin) for known profiles
  deviation <- 0
  for(i in 1:noKnown){
    #get sample info
    id <- testKnowns[i,idName]
    trueBin <- dataset[dataset[,idName] == id,'response']
    observedBin <- testKnowns[testKnowns[,idName] == id,'observedResponse']
    observedRank <- testKnowns[testKnowns[,idName] == id,'rank']
    #get rank distance of sample from true bin
    if(trueBin < observedBin){
      binDist <- abs(binRanks[trueBin+1] - observedRank - 1)
    } else if(trueBin > observedBin){
      binDist <- binRanks[trueBin] - observedRank
    } else {
      binDist <- 0
    }
    #add sample deviations together
    deviation <- deviation + abs(trueBin - observedBin)*binDist
  }
  return(deviation)
}